## Data Linking:  Universe with Price Volume

In [1]:
from datetime import datetime

import pandas as pd
import plotly.express as px
import polars as pl
import pyreadstat
import rapidfuzz as rf
from polars import col as c

In [2]:
# df = pl.read_parquet("/home/hzhang/data/russell_crsp_link_20250601.parquet")

In [3]:
# df.filter(c('Ticker').str.contains('^((LQ)|(CPLG))$') & c('Date').is_between(pl.date(2018,5,1),pl.date(2018,6,1)) )

### Load the data between needed dates

N.B. also check if known issue are fixed, and apply workaround if not
1. ⏳on FTSE: still > 100 missing weights in June 2014

In [4]:
df = pl.from_pandas(
    pyreadstat.read_file_multiprocessing(
        pyreadstat.read_sas7bdat,
        "/home/hzhang/data/wrds/russell/idx_holdings_us_20250531.sas7bdat",
        extra_date_formats=["YYMMDDN8"],
    )[0]
).filter(
    c("Date").is_between(
        datetime.fromisoformat("1995-05-31"),
        datetime.fromisoformat("2024-12-31"),
        closed="both",
    )
)

In [5]:
assert df.filter(
    c("Name").is_null()
).is_empty(), "rows with shifted columns not fixed https://wrds-support.wharton.upenn.edu/hc/en-us/requests/123032?"

In [6]:
if (
    max_num_missing_weight := df.filter(c("R3000_WT").is_null() | (c("R3000_WT") == 0))[
        "Date"
    ]
    .value_counts(sort=True)["count"]
    .max()
) > 100:
    print(
        f"ERROR: high {max_num_missing_weight=} pending https://wrds-support.wharton.upenn.edu/hc/en-us/requests/121314"
    )

ERROR: high max_num_missing_weight=175 pending https://wrds-support.wharton.upenn.edu/hc/en-us/requests/121314


In [7]:
df = df.with_columns(wt=c("R3000_WT").forward_fill().over("CUSIP"))

In [8]:
# df.filter(c('Date').dt.month_start() == datetime.fromisoformat('2014-06-01'))['wt'].sum()
df.filter(c("wt").is_null() | c("wt").eq(0))["Date"].value_counts().plot.line(
    "Date", "count"
)

alt.Chart(...)

In [9]:
df.filter(c("Ticker").is_in(["BYA", "AVB"])).filter(c("CUSIP").is_first_distinct())

Date,ConsCode,CUSIP,ISIN,Ticker,Exchange,Name,IndCodeICB,SupSecICB,SectorICB,SubSecICB,ShareChg,Shares,InvWeight,ValShares,GroShares,DefShares,DymShares,Return,MTDReturn,MktCap,Russell1000,Russell2000,Russell2500,RussellMC,RussellT200,RussellSC,RTOP50,R3000_WT,R3000G_WT,R3000V_WT,R1000_WT,R1000G_WT,R1000V_WT,R2000_WT,R2000G_WT,R2000V_WT,R2500_WT,R2500G_WT,R2500V_WT,RMIDC_WT,RMIDCG_WT,RMIDCV_WT,R200_WT,R200G_WT,R200V_WT,RSSC_WT,RSSCG_WT,RSSCV_WT,MktValue,ESCode,EconSector,SubSCode,SubSector,IndCode,Industry,SupSec,Sector,SubSec,Price,wt
date,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,f64,f64
1995-05-31,"""""","""072012107""","""""","""BYA""","""NYSE""","""BAY APARTMENT CMNTYS""","""""","""""","""""","""""",0.0,11544.0,null,8323.0,3221.0,null,null,null,null,null,"""N""","""Y""","""Y""","""N""","""N""","""N""","""""",0.000045,0.000024,0.000067,null,null,null,0.000466,0.000255,0.000684,0.000217,0.000118,0.000322,null,null,null,null,null,null,null,null,null,null,"""07""","""Materials & Processing""","""""","""""","""0000805""","""Real Estate""","""""","""""","""""",null,0.000045
1998-06-30,"""""","""053373106""","""""","""AVB""","""""","""AVALON BAY COMMUNITI""","""""","""""","""""","""""",1682.0,62123.0,null,62123.0,0.0,null,null,2.703,null,null,"""N""","""Y""","""Y""","""N""","""N""","""N""","""""",0.000224,null,0.000457,null,null,null,0.002273,null,0.004739,0.001184,null,0.00219,null,null,null,null,null,null,null,null,null,2.360674e6,"""10""","""Financial Services""","""""","""""","""0000806""","""Real Estate Investment Trusts …","""""","""""","""""",null,0.000224
1998-10-30,"""""","""053484101""","""""","""AVB""","""""","""AVALONBAY COMMS""","""""","""""","""""","""""",0.0,63574.0,null,63574.0,0.0,null,null,1.181,null,null,"""Y""","""N""","""Y""","""Y""","""N""","""N""","""""",0.000204,null,0.000424,0.000221,null,0.00046,null,null,null,0.001282,null,0.002306,0.000874,null,0.001404,null,null,null,null,null,null,2.0423e6,"""10""","""Financial Services""","""""","""""","""0000806""","""Real Estate Investment Trusts …","""""","""""","""""",null,0.000204


### Prune the data where the weight is zero or still missing

In [10]:
df = df.filter(c("wt").is_not_null() & (c("wt") > 0))

### Correct CUSIP mapping error

In [11]:
ticker_cusip_bdate_edate = [
    # Russell wrong, CRSP and Barra correct
    ["TNTX", "879244200", "", "1995-10-22"],
    # Russell wrong, Barra wrong, CRSP correct:
    ["TRFI", "89323N102", "1995-05-10", "1998-08-21"],
    # Rusell wrong, Barra and CRSP correct
    ["VWRX", "918435108", "1986-03-03", "1999-07-30"],
]

expr = pl
for ticker, cusip, bdate, edate in ticker_cusip_bdate_edate:
    bdate_dt = datetime.fromisoformat("1990-01-01" if bdate == "" else bdate)
    edate_dt = datetime.fromisoformat("2100-01-01" if edate == "" else edate)
    expr = expr.when(
        c("Ticker").eq(ticker)
        & c("Date")
        .dt.month_start()
        .is_between(bdate_dt.replace(day=1), edate_dt.replace(day=1))
    ).then(pl.lit(cusip))
expr = expr.otherwise("CUSIP").alias("CUSIP")

df = df.with_columns(expr)

### Load and also correct CRSP data

In [12]:
crsp_dse = pl.from_pandas(
    pyreadstat.read_sas7bdat(
        "/home/hzhang/data/wrds/crsp/stocknames_v2.sas7bdat",
        extra_date_formats=["YYMMDDN8"],
    )[0]
)

In [13]:
crsp_dse.filter(c("IssuerNm").str.contains("CORT BUS"))

PERMNO,PERMCO,NameDt,NameEndDt,SecurityBegDt,SecurityEndDt,HdrCUSIP,HdrCUSIP9,CUSIP,CUSIP9,Ticker,IssuerNm,PrimaryExch,ConditionalType,TradingStatusFlg,ShareClass,ShareType,SecurityType,SecuritySubType,USIncFlg,IssuerType,SICCD
f64,f64,date,date,date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64
82565.0,14142.0,1995-11-17,1995-12-20,1995-11-17,2000-02-17,"""22049310""","""220493100""","""22049310""","""220493100""","""CORT""","""CORT BUSINESS SERVICES CORP""","""Q""","""RW""","""A""","""""","""NS""","""EQTY""","""COM""","""Y""","""CORP""",6710.0
82565.0,14142.0,1995-12-21,1997-08-14,1995-11-17,2000-02-17,"""22049310""","""220493100""","""22049310""","""220493100""","""CBS""","""CORT BUSINESS SERVICES CORP""","""N""","""RW""","""A""","""""","""NS""","""EQTY""","""COM""","""Y""","""CORP""",7359.0
82565.0,14142.0,1997-08-15,2000-02-17,1995-11-17,2000-02-17,"""22049310""","""220493100""","""22049310""","""220493100""","""CBZ""","""CORT BUSINESS SERVICES CORP""","""N""","""RW""","""A""","""""","""NS""","""EQTY""","""COM""","""Y""","""CORP""",7359.0


In [14]:
crsp_dse = crsp_dse.with_columns(
    c("CUSIP9").replace(
        {
            # PAS dummy CUSIP
            "71399P90X": "71343P101",
            # XPRSA dummy CUSIP
            "90399N90X": "90338N103",
            # GNSA 144A
            "372450304": "372450106",
            # GNV 144A
            "372252304": "372252106",
            # ALW CUSIP upstream fix
            "020047205": "020047106",
        }
    )
)

In [15]:
crsp_dse.filter(c("PERMNO") == 80381).filter(c("CUSIP").is_first_distinct())

PERMNO,PERMCO,NameDt,NameEndDt,SecurityBegDt,SecurityEndDt,HdrCUSIP,HdrCUSIP9,CUSIP,CUSIP9,Ticker,IssuerNm,PrimaryExch,ConditionalType,TradingStatusFlg,ShareClass,ShareType,SecurityType,SecuritySubType,USIncFlg,IssuerType,SICCD
f64,f64,date,date,date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64
80381.0,30345.0,1994-03-11,1996-07-31,1994-03-11,2024-12-31,"""05348410""","""053484101""","""07201210""","""072012107""","""BYA""","""BAY APARTMENT COMMUNITIES INC""","""N""","""RW""","""A""","""""","""NS""","""EQTY""","""COM""","""Y""","""REIT""",1522.0
80381.0,30345.0,1998-06-05,1998-10-04,1994-03-11,2024-12-31,"""05348410""","""053484101""","""05337310""","""053373106""","""AVB""","""AVALON BAY COMMUNITIES INC""","""N""","""RW""","""A""","""""","""NS""","""EQTY""","""COM""","""Y""","""REIT""",6798.0
80381.0,30345.0,1998-10-05,2024-12-31,1994-03-11,2024-12-31,"""05348410""","""053484101""","""05348410""","""053484101""","""AVB""","""AVALONBAY COMMUNITIES INC""","""N""","""RW""","""A""","""""","""NS""","""EQTY""","""COM""","""Y""","""REIT""",6798.0


In [16]:
CUSIP9_date_s = [
    # CVTY wrong CUSIP change date
    ["222853103", 1998, 4, 1, "222862104", 1998, 4, 2],
    # HARB wrong CUSIP change date
    ["411510100", 1998, 3, 17, "411901101", 1998, 3, 18],
    # PPL wrong CUSIP change date
    ["709051106", 1995, 4, 30, "693499105", 1995, 5, 1],
]
expr_NameEndDt, expr_NameDt = pl, pl
for cusip1, y1, m1, d1, cusip2, y2, m2, d2 in CUSIP9_date_s:
    expr_NameEndDt = expr_NameEndDt.when(c("CUSIP9").eq(cusip1)).then(
        pl.date(y1, m1, d1)
    )
    expr_NameDt = expr_NameDt.when(c("CUSIP9").eq(cusip2)).then(pl.date(y2, m2, d2))
expr_NameEndDt = expr_NameEndDt.otherwise("NameEndDt").alias("NameEndDt")
expr_NameDt = expr_NameDt.otherwise("NameDt").alias("NameDt")
# expr_NameEndDt

In [17]:
crsp_dse = crsp_dse.with_columns(expr_NameEndDt, expr_NameDt)

### Get rid of "_x" dummies

In [18]:
df = df.filter(~c("Ticker").str.contains("_x"))

## Join R3K universe with CRSP coverage

In [19]:
# 0: if month-end falls in between CRSP dates
# 1: if month falls in between CRSP months
# 2: if month falls in between lenient CRSP months

res0 = df.join(crsp_dse, left_on="CUSIP", right_on="CUSIP9").with_columns(
    match_priority=pl.when(c("Date").is_between(c("NameDt"), c("NameEndDt")))
    .then(0)
    .when(
        c("Date")
        .dt.month_start()
        .is_between(c("NameDt").dt.month_start(), c("NameEndDt").dt.month_start())
    )
    .then(1)
    .when(
        c("Date").is_between(
            c("NameDt").dt.month_start().dt.offset_by("-1mo"),
            c("NameEndDt").dt.month_start().dt.offset_by("6mo"),
        )
    )
    .then(2)
    .otherwise(-1)
)

In [20]:
res = (
    res0.filter(c("match_priority") >= 0)
    .sort(["Date", "PERMNO", "match_priority"])
    .unique(subset=["Date", "PERMNO"], keep="first", maintain_order=True)
)

### Verify results
1. Unmatched R3K consituents are foreign stocks (not covered by CRSP), or stock traded on Chicago Philadelpha exchanges historically
2. match_priority > 0 should be rare

In [21]:
df.join(
    res,
    how="anti",
    on=["Date", "CUSIP"],
).filter(
    ~c("Ticker").is_in(["AFE", "AX", "GBC"])  # in other exchanges
    & ~c("Ticker").str.ends_with("x")  # Known dummy tickers
).select(["Date", "ConsCode", "CUSIP", "ISIN", "Ticker", "Exchange", "Name"]).filter(
    c("CUSIP").is_first_distinct()
)

Date,ConsCode,CUSIP,ISIN,Ticker,Exchange,Name
date,str,str,str,str,str,str
1995-05-31,"""""","""206189102""","""""","""CEFT""","""NASDAQ""","""CONCORD EFS INC"""
2007-06-29,"""""","""G5299G106""","""""","""VLCCF""","""""","""KNIGHTSBRIDGE TANKERS"""
2007-06-29,"""""","""P16994132""","""""","""BLX""","""""","""BANCO LATINOAMERICANO"""
2008-06-30,"""""","""G02995101""","""""","""ASI""","""""","""AMERICAN SAFETY INS HLDG"""
2023-03-31,"""C69803""","""45031U101""","""US45031U1016""","""STAR""","""NYSE""","""ISTAR INC"""
2023-12-29,"""C196349""","""14817C115""","""US14817C1154""","""SAVAW""","""NASDAQ""","""CASAVA SCIENCES WARRANTS"""


In [22]:
res.filter(c("match_priority") == 1)["Date"].value_counts(sort=True).plot.line(
    x="Date", y="count"
)

alt.Chart(...)

In [29]:
res.filter(c('Ticker').str.contains('^CB(S|Z)$') & c('Date').is_between(pl.date(1997,7,1), pl.date(1997,10,1)))

Date,ConsCode,CUSIP,ISIN,Ticker,Exchange,Name,IndCodeICB,SupSecICB,SectorICB,SubSecICB,ShareChg,Shares,InvWeight,ValShares,GroShares,DefShares,DymShares,Return,MTDReturn,MktCap,Russell1000,Russell2000,Russell2500,RussellMC,RussellT200,RussellSC,RTOP50,R3000_WT,R3000G_WT,R3000V_WT,R1000_WT,R1000G_WT,R1000V_WT,R2000_WT,R2000G_WT,R2000V_WT,…,RSSC_WT,RSSCG_WT,RSSCV_WT,MktValue,ESCode,EconSector,SubSCode,SubSector,IndCode,Industry,SupSec,Sector,SubSec,Price,wt,PERMNO,PERMCO,NameDt,NameEndDt,SecurityBegDt,SecurityEndDt,HdrCUSIP,HdrCUSIP9,CUSIP_right,Ticker_right,IssuerNm,PrimaryExch,ConditionalType,TradingStatusFlg,ShareClass,ShareType,SecurityType,SecuritySubType,USIncFlg,IssuerType,SICCD,match_priority
date,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,date,date,date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,i32
1997-07-31,"""""","""220493100""","""""","""CBS""","""NYSE""","""CORT BUSINESS SERVIC""","""""","""""","""""","""""",0.0,12788.0,null,1778.0,11010.0,null,null,null,null,null,"""N""","""Y""","""Y""","""N""","""N""","""N""","""""",0.00005,0.000085,0.000014,null,null,null,0.000521,0.000906,0.000143,…,null,null,null,431595.0,"""10""","""Financial Services""","""""","""""","""0000808""","""Rental & Leasing Services: Com…","""""","""""","""""",null,0.00005,82565.0,14142.0,1995-12-21,1997-08-14,1995-11-17,2000-02-17,"""22049310""","""220493100""","""22049310""","""CBS""","""CORT BUSINESS SERVICES CORP""","""N""","""RW""","""A""","""""","""NS""","""EQTY""","""COM""","""Y""","""CORP""",7359.0,0
1997-08-29,"""""","""220493100""","""""","""CBZ""","""NYSE""","""CORT BUSINESS SERVIC""","""""","""""","""""","""""",0.0,12788.0,null,1778.0,11010.0,null,null,null,null,null,"""N""","""Y""","""Y""","""N""","""N""","""N""","""""",0.000059,0.000101,0.000016,null,null,null,0.000577,0.000989,0.000161,…,null,null,null,489940.25,"""10""","""Financial Services""","""""","""""","""0000808""","""Rental & Leasing Services: Com…","""""","""""","""""",null,0.000059,82565.0,14142.0,1997-08-15,2000-02-17,1995-11-17,2000-02-17,"""22049310""","""220493100""","""22049310""","""CBZ""","""CORT BUSINESS SERVICES CORP""","""N""","""RW""","""A""","""""","""NS""","""EQTY""","""COM""","""Y""","""CORP""",7359.0,0
1997-09-30,"""""","""220493100""","""""","""CBZ""","""NYSE""","""CORT BUSINESS SERVIC""","""""","""""","""""","""""",0.0,12788.0,null,1778.0,11010.0,null,null,null,null,null,"""N""","""Y""","""Y""","""N""","""N""","""N""","""""",0.000058,0.000101,0.000016,null,null,null,0.000563,0.000956,0.000159,…,null,null,null,510720.75,"""10""","""Financial Services""","""""","""""","""0000808""","""Rental & Leasing Services: Com…","""""","""""","""""",null,0.000058,82565.0,14142.0,1997-08-15,2000-02-17,1995-11-17,2000-02-17,"""22049310""","""220493100""","""22049310""","""CBZ""","""CORT BUSINESS SERVICES CORP""","""N""","""RW""","""A""","""""","""NS""","""EQTY""","""COM""","""Y""","""CORP""",7359.0,0


## Write results

In [30]:
res.write_parquet("/home/hzhang/data/russell_crsp_link_20250619.parquet")

In [24]:
raise InterruptedError

InterruptedError: 

In [ ]:
# FTSE won't fix https://wrds-support.wharton.upenn.edu/hc/en-us/requests/120961
df = df.with_columns(
    Ticker=pl.when(
        (c("Name") == "US FOODSERVICE")
        & (c("Date") < datetime.fromisoformat("1998-06-01"))
    )
    .then(pl.lit("UFS"))
    .otherwise(c("Ticker"))
)

In [ ]:
# px.line(df.filter(c('MktValue').is_null()).group_by('Date', maintain_order=True).len(), x='Date', y='len', markers=True)

### Get rid of rows with shifted columns (2025-03-26)
filed WRDS [ticket](https://wrds-support.wharton.upenn.edu/hc/en-us/requests/123032)

In [ ]:
df = df.filter(~c("Name").is_null())  # Known bug filed with WRDS

### Fix missing R3000_WT columns

Among MarketCap, R3000_WT, and MarketVal, choose the one column with least missing data. Then just fill forward.

⏳Per WRDS, FTSE sent them new data, which they will populate in a week. [link](https://wrds-support.wharton.upenn.edu/hc/en-us/requests/121314)

In [ ]:
df = df.with_columns(wt=c("R3000_WT").forward_fill().over("CUSIP")).with_columns(
    wt=pl.when(c("Date").lt(datetime.fromisoformat("1998-04-01")))
    .then(c("wt") / 100.0)
    .otherwise(c("wt"))
)

### Rid of zero or missing weights

In [ ]:
px.line(
    df.filter(c("wt").is_null() | (c("wt") == 0))
    .group_by("Date", maintain_order=True)
    .agg(pl.len()),
    x="Date",
    y="len",
    markers=True,
    title="# zero or null weights",
)

In [ ]:
df = df.filter(c("wt").is_not_null() & (c("wt") > 0))

In [ ]:
# with pl.Config(tbl_rows=-1):
#     display(df.filter((c('wt')==0 ) & (c('Date').dt.month_start() == datetime.fromisoformat('2001-04-01'))))

### Hand fix a early ticker error

⏳[ticket](https://wrds-support.wharton.upenn.edu/hc/en-us/requests/120961) to WRDS -> Russell



In [ ]:
df = df.with_columns(
    Ticker=pl.when(
        (c("Name") == "US FOODSERVICE")
        & (c("Date") < datetime.fromisoformat("1998-06-01"))
    )
    .then(pl.lit("UFS"))
    .otherwise(c("Ticker"))
)

## Hand fix CUSIPs

### Non-NY Exchanges

Examples are Chicago, Philadelpha, Pacific, etc. Exchanges.

1. American Financial Enterprises (AFEP, AFEM) prices are not available because they were traded in Chicago and Pacific exchanges thus not covered by CRSP (SEC [link](https://www.sec.gov/Archives/edgar/data/319157/0000319157-96-000007.txt))
2. Archipelago Holdings (AX) trades on Pacific Exchange and not covered (SEC [link](https://www.sec.gov/Archives/edgar/data/1107389/000104746905007391/a2154055z10-k.htm))
3. Greif Brothers traded on Chicago exchange for much of 1995 (SEC [link](https://www.sec.gov/Archives/edgar/data/43920/0000043920-96-000001.txt))

### Erronerous early CUSIPS
There was a number of CUSIP erros prior to 2000, which is fixed using CRSP as ground truth.

### "_x" suffix
A number of recent tickers have "_x" and CUSIPs have "XX" in them, ✅raised to WRDS in turn to FTSE ([link](https://wrds-support.wharton.upenn.edu/hc/en-us/requests/121321))
> Please note the _x and xxxx in CUSIP/ISIN are used to represent temporary lines representing a stock distribution or some other corporate action.
> 
> Section 3
>
> The use of dummy lines in FTSE
>
> Russell indices
> 
> The use of dummy lines in FTSE Russell indices
>
> 3.1 Dummy lines are non-tradable instruments which have been temporarily created by FTSE Russell in order
to reflect a corporate event.
> 
> 3.2 The use of dummy lines is normally determined on an ad hoc basis and typically results from complexities
surrounding a corporate event.
>
> 3.3 Where the use of dummy lines is necessary FTSE Russell provides advance notification either via an
informative notice published on the FTSE Russell website, or via the Russell Corporate Action Calendar.
Dummy lines will also be visible within the standard corporate action deliverables when they are being
utilized.
>
> 3.4 Dummy lines are generally used in order to ensure the index reflects the investor experience or in order to
facilitate index replication by index funds.

In [ ]:
df = df.with_columns(CUSIP8=c("CUSIP").str.head(8))

on_other_exchanges = ["AFE", "AX", "GBC"]

ticker_cusip8_bdate_edate = [
    [
        "ALW",
        "02004710",
        "",
        "1997-07-31",
    ],  # CUSIP fixed it, Barra is correct, CRSP was wrong,
    ["GNV", "37225230", "", "2000-01-01"],  # WAITING ON CRSP: CRSP lists a 144A
    ["TNTX", "87924420", "", "1995-10-22"],  # russell wrong, CRSP and Barra correct
    ["TNTX", "87259710", "1995-10-23", "2004-03-04"],  # ibid
    [
        "TRFI",
        "89323N10",
        "1995-05-10",
        "1998-08-21",
    ],  # russell wrong, Barra wrong, CRSP correct
    [
        "VWRX",
        "91843510",
        "1986-03-03",
        "1999-07-30",
    ],  # Rusell wrong, Barra and CRSP correct
    ["PAS", "71399P90", "", "2000-11-30"],  # dummy cusip: CRSP assigned dummy CUSIP
    ["PAS", "71343P20", "2001-01-24", "2010-02-26"],
    [
        "GNSA",
        "37245030",
        "1997-04-22",
        "1999-06-17",
    ],  # WAITING CRSP: see CRSP erroneously uses a 144A CUSIP
    [
        "CVTY",
        "22285310",
        "",
        "1999-06-30",
    ],  #  WAITING CRSP: see CRSP got the dates wrong of change of CUSIP
    [
        "HARB",
        "41190110",
        "1997-06-26",
        "2006-11-30",
    ],  #  WAITING CRSP: see CRSP got the dates wrong of change of CUSIP
    [
        "PPL",
        "70905110",
        "",
        "1997-09-11",
    ],  # TO FILE: Barra got the dates wrong of change of CUSIP
    [
        "XPRSA",
        "90399N90",
        "1994-10-16",
        "2007-10-12",
    ],  # dummy cusip: CRP assigned dummy CUSIP
]

for ticker, cusip8, bdate, edate in ticker_cusip8_bdate_edate:
    bdate_dt = datetime.fromisoformat("1990-01-01" if bdate == "" else bdate)
    edate_dt = datetime.fromisoformat("2100-01-01" if edate == "" else edate)
    df = df.with_columns(
        CUSIP8=pl.when(
            c("Ticker").eq(ticker) & c("Date").is_between(bdate_dt, edate_dt)
        )
        .then(pl.lit(cusip8))
        .otherwise(c("CUSIP8"))
    )

In [ ]:
df.filter(c("Ticker") == "GNV")

In [ ]:
# df.filter(df.select(["Date", "Ticker"]).is_duplicated())

In [ ]:
crsp_dse = pl.from_pandas(
    pd.read_sas("/home/hzhang/data/wrds/dsenames.sas7bdat", encoding="latin-1")
).with_columns(c("NAMEDT").cast(pl.Date), c("NAMEENDT").cast(pl.Date))

In [ ]:
crsp_dse.write_parquet

In [ ]:
crsp_dse.filter(c("COMNAM").str.starts_with("VARITY"))

In [ ]:
# df.filter(c("Ticker")=='HARB').filter(c('CUSIP').is_first_distinct())

In [ ]:
# crsp_dse.filter(c("TICKER") == 'PPL')
with pl.Config(tbl_rows=-1):
    display(crsp_dse.filter(c("PERMCO") == 21376))

In [ ]:
crsp_dse.filter(c("COMNAM").str.starts_with("PEPSIAM"))

In [ ]:
crsp_dse.filter(c("NCUSIP").str.head(6) == "71343P")

In [ ]:
crsp_dse.filter(c("NCUSIP").str.starts_with("90338N"))

In [ ]:
crsp_dse.join(crsp_dse, on=c("NCUSIP").str.head(6), how="inner").filter(
    c("PERMCO") != c("PERMCO_right")
)

In [ ]:
def cusip8_to_9(
    df: pl.DataFrame, cusip8_col: str = "CUSIP8", cusip9_col: str = "CUSIP9"
) -> pl.DataFrame:
    # Assume df is a Polars DataFrame with a column "cusip8" containing 8-char CUSIP strings
    df_o = (
        df.with_columns(
            # 1. Ensure CUSIP is uppercase (if not already) to handle letters uniformly
            pl.col(cusip8_col)
            .str.to_uppercase()
            .alias(cusip8_col)
        )
        .with_columns(
            # 2. Compute numeric value of each character (val0...val7 for positions 0-7)
            *[
                pl.when(pl.col(cusip8_col).str.slice(i, 1).str.contains(r"^\d$"))
                .then(
                    pl.col(cusip8_col).str.slice(i, 1).str.to_integer(strict=False)
                )  # digit -> its numeric value
                .when(pl.col(cusip8_col).str.slice(i, 1).str.contains(r"^[A-Z]$"))
                .then(
                    pl.col(cusip8_col)
                    .str.slice(i, 1)
                    .map_elements(lambda x: ord(x), return_dtype=pl.UInt8)
                    - 55
                )  # letter -> codepoint minus 55 (e.g. 'A'(65) -> 10)
                .when(pl.col(cusip8_col).str.slice(i, 1) == "*")
                .then(pl.lit(36))  # '*' -> 36
                .when(pl.col(cusip8_col).str.slice(i, 1) == "@")
                .then(pl.lit(37))  # '@' -> 37
                .when(pl.col(cusip8_col).str.slice(i, 1) == "#")
                .then(pl.lit(38))  # '#' -> 38
                .otherwise(pl.lit(0))
                .alias(f"val{i}")
                for i in range(8)
            ]
        )
        .with_columns(
            # 3. Sum the weighted values with the Luhn double-add-double rules
            pl.sum_horizontal(
                [
                    # For even index (0-based) positions: use the value as-is
                    (
                        pl.col(f"val{i}")
                        if i % 2 == 0
                        # For odd index positions: double the value and sum its digits => (val*2 mod 10) + (val*2 // 10)
                        else (
                            (pl.col(f"val{i}") * 2) % 10 + (pl.col(f"val{i}") * 2) // 10
                        )
                    )
                    for i in range(8)
                ]
            ).alias("sum")
        )
        .with_columns(
            # 4. Compute check digit = (10 - (sum mod 10)) mod 10
            ((10 - (pl.col("sum") % 10)) % 10).alias("check_digit")
        )
        .with_columns(
            # 5. Concatenate original 8-char CUSIP with the check digit to form CUSIP-9
            (pl.col(cusip8_col) + pl.col("check_digit").cast(pl.Utf8)).alias(cusip9_col)
        )
    )

    # (Optionally, drop intermediate columns if they are not needed beyond this point)
    df_o = df_o.drop([f"val{i}" for i in range(8)] + ["sum", "check_digit"])
    return df_o

In [ ]:
crsp_dse_cusip9 = cusip8_to_9(
    crsp_dse, cusip8_col="NCUSIP", cusip9_col="NCUSIP9"
).select("PERMNO", "NCUSIP9", pl.exclude("PERMNO", "NCUSIP9"))

In [ ]:
crsp_dse_cusip9.filter(c("NCUSIP").str.starts_with("372252"))

In [ ]:
df.filter(c("CUSIP").str.head(8).is_in(["02004710", "02004720"])).filter(
    c("CUSIP").is_first_distinct()
)

In [ ]:
raise InterruptedError

In [ ]:
df.filter(c("CUSIP8") == "87259710")

In [ ]:
res0 = df.join(crsp_dse, left_on="CUSIP8", right_on="NCUSIP").with_columns(
    match_priority=pl.when(
        c("Date")
        .dt.month_start()
        .is_between(c("NAMEDT").dt.month_start(), c("NAMEENDT").dt.month_start())
    )
    .then(0)
    .otherwise(
        pl.when(
            c("Date").is_between(
                c("NAMEDT").dt.month_start().dt.offset_by("-1mo"),
                c("NAMEENDT").dt.month_start().dt.offset_by("6mo"),
            )
        )
        .then(1)
        .otherwise(-1)
    )
)

In [ ]:
res0.filter(c("CUSIP") == "888741105")

In [ ]:
# res0.filter((c('PERMNO')==88661) & (c('Date').dt.month_start() == datetime.fromisoformat('2017-10-01')))

In [ ]:
res = (
    res0.filter(c("match_priority") >= 0)
    .sort(["Date", "PERMNO", "match_priority"])
    .unique(subset=["Date", "PERMNO"], keep="first", maintain_order=True)
)

In [ ]:
# res.filter((c('Ticker')=='TPR') & (c('Date').dt.month_start() == datetime.fromisoformat('2017-10-01')))

In [ ]:
df.join(
    res,
    how="anti",
    on=["Date", "CUSIP8"],
).filter(
    ~c("Ticker").is_in(on_other_exchanges)
    & ~c("Ticker").str.ends_with("x")  # Known dummy tickers
).select(["Date", "ConsCode", "CUSIP", "ISIN", "Ticker", "Exchange", "Name"])

In [ ]:
px.line(
    res.filter(c("match_priority") == 1)
    .group_by("Date", maintain_order=True)
    .agg(pl.len()),
    x="Date",
    y="len",
    markers=True,
    title="# non-exact matches ",
)

In [ ]:
res.write_parquet("/home/hzhang/data/russell_crsp_link.parquet")

In [ ]:
res["CUSIP8"].unique().count()

In [ ]:
raise InterruptedError

In [ ]:
res.filter(c("CUSIP") == "887315109")

In [ ]:
(
    res.with_columns(month_no=c("Date").dt.year() * 12 + c("Date").dt.month())
    .select(["Date", "PERMNO", "month_no"])
    .with_columns(diff_month_no=c("month_no").diff().over("PERMNO"))
    .filter(c("diff_month_no") > 1)
).filter(c("diff_month_no") == 2)

In [ ]:
res.filter(
    c("PERMNO").eq(16465)
    & c("Date").is_between(
        datetime.fromisoformat("2020-07-01"), datetime.fromisoformat("2020-10-01")
    )
)

In [ ]:
[c for c in res.columns if c.startswith("R")]

In [ ]:
res["PERMNO"].unique()

In [ ]:
# df.filter(c('Ticker').is_in(on_other_exchanges))
df.filter(c("Ticker") == "TRFI")

In [ ]:
df.filter(c("Date") < datetime.fromisoformat("2000-01-01"))["Exchange"].unique()

In [ ]:
crsp_dse.filter(c("NCUSIP").eq("25380710"))

In [ ]:
crsp_dse.filter(c("TICKER").eq("USITQ"))

In [ ]:
crsp_dse.filter(c("COMNAM").str.starts_with("GREIF"))

In [ ]:
rf.process.extractOne(
    "ARCHIPELEGO HOLDINGS INC",
    crsp_dse["COMNAM"].unique(),
    scorer=rf.fuzz.ratio,
    processor=rf.utils.default_process,
)

In [ ]:
import rapidfuzz as rf

In [ ]:
choices = ["Atlanta Falcons", "New York Jets", "New York Giants", "Dallas Cowboys"]

In [ ]:
rf.process.extractOne(
    "cowboys", choices, scorer=rf.fuzz.ratio, processor=rf.utils.default_process
)

In [ ]:
rf.process.extractOne(
    "ADLAC",
    crsp_dse["TICKER"].unique(),
    scorer=rf.fuzz.ratio,
    processor=rf.utils.default_process,
)

In [ ]:
rf.process.extractOne(
    "ADELPHIA COMMUN -CL",
    crsp_dse["COMNAM"].unique(),
    scorer=rf.fuzz.ratio,
    processor=rf.utils.default_process,
)

In [ ]:
[
    df.filter(c("Date").dt.month_start() == datetime.fromisoformat(f"{month}-01"))[
        "wt"
    ].sum()
    for month in ["2012-07", "2014-06"]
]

In [ ]:
px.line(
    df.group_by("Date").agg(c("wt").sum()).sort(by="Date"),
    x="Date",
    y="wt",
    markers=True,
)

In [ ]:
df

In [ ]:
crsp_dse